In [1]:
import logging
import threading
import random
import sys

from multiprocessing import Process
from time import sleep
from typing import Optional

logging.basicConfig()

from kazoo.client import KazooClient
from kazoo.recipe.watchers import DataWatch

ACTION_COMMIT =   b'commit'
ACTION_ROLLBACK = b'rollback'
ACTION_DISCONNECT = b'one of the participants disconnected'

WAIT_HARD_WORK_SECONDS = 20
zk_list = list()
p_list = list()

In [2]:
class Client(Process):
    def __init__(self, root: str, id: int, zk):
        super().__init__()
        self.url = f'{root}/{id}'
        self.root = root
        self.id = id
        self.zk = zk
             
    def run(self):
        #@zk.DataWatch(self.url)
        def watch_myself(data, stat):
            if data == ACTION_DISCONNECT:
                print(f'Клиент {self.id} уведомлён о том, что один из участников отключился')
            else:
                if(stat.version == 1):
                    sleep(1)
                if stat.version != 0:
                    print(f'Клиент {self.id} принял решение от координатора: {data.decode()}')
            
        self.zk.start()
        
        value = ACTION_COMMIT if random.random() > 0.5 else ACTION_ROLLBACK
        print(f'Клиент {self.id} запросил {value.decode()}')
        self.zk.create(self.url, value, ephemeral=True)
        datawatcher = DataWatch(self.zk, self.url, watch_myself)
        
        sleep(WAIT_HARD_WORK_SECONDS)
        self.zk.stop()
        print(f'Клиент {self.id} отключился')
        self.zk.close()


class Coordinator:

    timer: Optional[threading.Timer] = None

    @staticmethod
    def main(number_of_clients = 3, duration = 3):
        Coordinator.session_logs = [False] * number_of_clients # Храним, заходил ли уже клиент
        coordinator = KazooClient()
        coordinator.start()

        if coordinator.exists('/task_2'):
            coordinator.delete('/task_2', recursive=True)

        coordinator.create('/task_2')
        coordinator.create('/task_2/transaction')

        Coordinator.timer = None
        
        def make_decision():
            Coordinator.timer.cancel()
            tr_clients = coordinator.get_children('/task_2/transaction')
            commit_counter = 0
            abort_counter = 0
            for client in tr_clients:
                commit_counter += int(coordinator.get(f'/task_2/transaction/{client}')[0] == ACTION_COMMIT)
                abort_counter +=  int(coordinator.get(f'/task_2/transaction/{client}')[0] == ACTION_ROLLBACK)

            # Принимает commit только единогласно
            final_action = ACTION_COMMIT if commit_counter == number_of_clients else ACTION_ROLLBACK
            for client in tr_clients:
                coordinator.set(f'/task_2/transaction/{client}', final_action) # Рассылаем результат
                
        def true_check_clients():
            tr_clients = coordinator.get_children('/task_2/transaction')
            for i in range(len(Coordinator.session_logs)):
                if Coordinator.session_logs[i] is True and str(i) not in tr_clients:
                    print("Один участник отключился, всем остальным разослано сообщение с оповещением")
                    sleep(0.5)
                    Coordinator.timer.cancel()
                    for client in tr_clients:
                        coordinator.set(f'/task_2/transaction/{client}', ACTION_DISCONNECT)
                    sleep(0.5)
                    for client in tr_clients:
                        zk_list[int(client)].stop()
                        zk_list[int(client)].close()
                        p_list[int(client)].kill()
                    Coordinator.timer.cancel()
                    sys.exit()

        @coordinator.ChildrenWatch('/task_2/transaction')
        def watch_clients(clients):
            for client in clients:
                Coordinator.session_logs[int(client)] = True
                
            if len(clients) == 0:
                if Coordinator.timer is not None:
                    Coordinator.timer.cancel()
            else:
                if Coordinator.timer is not None:
                    Coordinator.timer.cancel()
                Coordinator.timer = threading.Timer(duration, true_check_clients) # Проверяем, не отключился ли клиент
                Coordinator.timer.daemon = True
                Coordinator.timer.start()

            if len(clients) < number_of_clients:
                print(f'Подключенные клиенты:{clients}')
            elif len(clients) == number_of_clients:
                make_decision()

        root = '/task_2/transaction'
        
        for i in range(number_of_clients):
            zk_list.append(KazooClient())
            p = Client(root, i, zk_list[-1])
            p_list.append(p)
            p.start()
            sleep(7)
            
Coordinator.main()

Подключенные клиенты:[]
Клиент 0 запросил rollback
Подключенные клиенты:['0']
Клиент 1 запросил rollback
Подключенные клиенты:['0', '1']
Клиент 2 запросил rollback
Клиент 0 принял решение от координатора: rollbackКлиент 1 принял решение от координатора: rollback
Клиент 2 принял решение от координатора: rollback

Клиент 0 отключился
Подключенные клиенты:['1', '2']
Один участник отключился, всем остальным разослано сообщение с оповещением
Клиент 1 уведомлён о том, что один из участников отключилсяКлиент 2 уведомлён о том, что один из участников отключился

Подключенные клиенты:[]
